In [1]:
import numpy as np
import pandas as pd

from datetime import datetime

In [2]:
def read_df(fname):
    df = pd.read_csv('companies.csv', parse_dates=['branch_issue_deadline', 'exam_date', 'interview_date', 'ppt_date', 'updated_at', 'willingness_deadline'])
    return df

In [3]:
def query_dept_dates(df, dept, date_col, min_date=None):
    if min_date is None:
        min_date = datetime.now()
    return df[(df[dept] == 1) & (df[date_col] > min_date)].copy()

In [4]:
def create_notification_df(df, dept, date_col, min_date=None, suffix=None):
    notification_df = query_dept_dates(df, dept, date_col, min_date)
    
    if suffix is None:
        suffix = date_col
    notification_df.loc[:, 'email_msg_{}'.format(suffix)] = create_emails(notification_df)
    notification_df.loc[:, 'email_count_{}'.format(suffix)] = 0
    return notification_df.reset_index(drop=True)

In [5]:
def email(x):
    company = x['company_name']
    package_ctc = x['idd_imd_ctc'] / 10**5
    package_home = x['idd_imd_home'] / 10**5
    willingness = x['willingness_deadline']
    msg = ('Willingness for {} is open till {}. '
           'The Company is offering a CTC of {:.2f} Lakhs and Take Home of {:.2f} Lakhs'
           .format(company, willingness, package_ctc, package_home))
    return msg

In [6]:
def create_emails(df):
    return df.apply(email, axis=1)

In [13]:
df = read_df('companies.csv')
willingness_df = create_notification_df(df, 'mat', 'willingness_deadline', suffix='willingness')
willingness_df.loc[:, ['company_name', 'email_msg_willingness']]

,company_name,email_msg_willingness
0,AXTRIA,Willingness for AXTRIA is open till 2017-09-27...
1,Media.net,Willingness for Media.net is open till 2017-09...
2,Flock,Willingness for Flock is open till 2017-09-27 ...
3,WIPRO,Willingness for WIPRO is open till 2017-09-27 ...


In [24]:
willingness_df.apply(lambda x: '{} willingness deadline'.format(x['company_name']), axis=1)

0       AXTRIA willingness deadline
1    Media.net willingness deadline
2        Flock willingness deadline
3        WIPRO willingness deadline
dtype: object

In [26]:
list(zip(pd.Series([1, 2, 3, 4]), pd.Series([1, 2, 3, 4])))

[(1, 1), (2, 2), (3, 3), (4, 4)]

In [8]:
willingness_df.to_csv('willingness.csv', index=False)
pd.read_csv('willingness.csv')

,active_backlog,branch_issue_deadline,btech_ctc,btech_home,cgpa,company_name,exam_date,idd_imd_ctc,idd_imd_home,interview_date,...,bce,chy,phy,bme,btech,idd,mtech,phd,email_msg_willingness,email_count_willingness
0,0.0,2017-09-01 11:00:00,NaN,NaN,6.0,AXTRIA,NaN,780000.0,NaN,NaN,...,1,1,1,1,0,1,1,0,Willingness for AXTRIA is open till 2017-09-27...,0
1,0.0,2017-07-11 23:00:00,2500000.0,1500000.0,0.0,Media.net,2017-10-08 00:00:00,2500000.0,1500000.0,NaN,...,1,1,1,1,1,1,1,0,Willingness for Media.net is open till 2017-09...,0
2,0.0,2017-07-11 23:00:00,2500000.0,1500000.0,0.0,Flock,2017-10-08 00:00:00,2500000.0,1500000.0,NaN,...,1,1,1,1,1,1,1,0,Willingness for Flock is open till 2017-09-27 ...,0
3,0.0,2017-08-09 13:00:00,900000.0,NaN,7.0,WIPRO,NaN,900000.0,NaN,NaN,...,1,1,1,1,1,1,0,0,Willingness for WIPRO is open till 2017-09-27 ...,0
